In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys

# TODO: change this to the path to your homework folder
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '487/Project/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
os.chdir(GOOGLE_DRIVE_PATH)

MessageError: ignored

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import transforms
from torchvision.models import vgg19
from PIL import Image
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
# Define the dataset class to load images and labels
class MyDataset(Dataset):
    def __init__(self, data_path, label_path, transform=None):
        label = np.loadtxt(label_path)
        self.labels = torch.from_numpy(label)[:2500]

        self.transform = transform

        self.data = ()
        for i in range(len(self.labels)):
            if i % 100 == 0:
              print(i)
            img_path = data_path + f"{i+1}.jpg"
            # self.data.append(img_path)
            img = Image.open(img_path).convert('RGB')
            self.data = self.data + (self.transform(img),)
            img.close()
        self.data = torch.stack(self.data)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        label = self.labels[index]

        # img_path = self.data[index]
        # mimg = Image.open(img_path).convert('RGB')
        # img = self.transform(mimg)
        # mimg.close()
        img = self.data[index]
        return img, label

In [ ]:
# Define the transforms for data augmentation
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = MyDataset(data_path='train_images/',
                          label_path='label/train_labels.txt',
                          transform=data_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400


In [ ]:
model = vgg19(pretrained=True)

# model.classifier[6] = nn.Linear(4096, 200)
for param in model.features.parameters():
    param.requires_grad = False
model.classifier.add_module('7', nn.ReLU())
model.classifier.add_module('8', nn.Linear(1000, 800))
model.classifier.add_module('9', nn.ReLU())
model.classifier.add_module('10', nn.Linear(800, 5))


# Define the loss function and optimizer
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
model.to('cuda')
num_epoch = 40
for epoch in range(num_epoch):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (images, labels) in enumerate(train_loader):
        # print(i)
        images = images.to('cuda')
        labels = labels.to('cuda')        
        
        # update the model parameters using backpropagation
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs.float(), labels.float())

        loss.backward()
        optimizer.step()
        
        # update running loss
        running_loss += loss.item()
        
        # update accuracy
        predicted = torch.round(outputs)
        # predicted = torch.where((outputs == torch.max(outputs, dim=1, keepdim=True)[0]), torch.tensor(1).cuda(), torch.tensor(0).cuda())

        total += labels.size(0) * labels.size(1)
        correct += (predicted == labels).sum().item()
        
    # compute average loss and accuracy for epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100.0 * correct / total
    
    # print progress
    print(f"Epoch [{epoch+1}/{num_epoch}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

Epoch [1/40], Loss: 1.6004, Accuracy: 56.60%
Epoch [2/40], Loss: 0.5235, Accuracy: 60.36%
Epoch [3/40], Loss: 0.4781, Accuracy: 62.32%
Epoch [4/40], Loss: 0.4400, Accuracy: 63.61%
Epoch [5/40], Loss: 0.4194, Accuracy: 65.58%
Epoch [6/40], Loss: 0.3962, Accuracy: 66.90%
Epoch [7/40], Loss: 0.3835, Accuracy: 67.11%
Epoch [8/40], Loss: 0.3619, Accuracy: 69.00%
Epoch [9/40], Loss: 0.3385, Accuracy: 70.67%
Epoch [10/40], Loss: 0.3286, Accuracy: 70.70%
Epoch [11/40], Loss: 0.3115, Accuracy: 73.14%
Epoch [12/40], Loss: 0.2912, Accuracy: 73.84%
Epoch [13/40], Loss: 0.2795, Accuracy: 75.63%
Epoch [14/40], Loss: 0.2542, Accuracy: 76.89%
Epoch [15/40], Loss: 0.2426, Accuracy: 77.30%
Epoch [16/40], Loss: 0.2286, Accuracy: 78.39%
Epoch [17/40], Loss: 0.2109, Accuracy: 80.09%
Epoch [18/40], Loss: 0.2038, Accuracy: 80.07%
Epoch [19/40], Loss: 0.1956, Accuracy: 81.51%
Epoch [20/40], Loss: 0.1845, Accuracy: 81.98%
Epoch [21/40], Loss: 0.1736, Accuracy: 82.90%
Epoch [22/40], Loss: 0.1672, Accuracy: 83.2

In [ ]:
new_model = nn.Sequential(*list(model.classifier.children())[:-2])

In [ ]:
torch.save(new_model.state_dict(), 'model_weights.pt')

In [ ]:
new_model = nn.Sequential(*list(model.children())[:-1])

NameError: ignored

In [ ]:
img_path = 'train_images/1.jpg'
img = Image.open(img_path).convert('RGB')
embed = new_model(data_transform(img).cuda())
print(embed.size())

torch.Size([512, 7, 7])


In [ ]:
data_path = 'train_images/'
output = ()
for i in range(7000):
  if i % 100 == 0:
    print(i)
  img_path = data_path + f"{i+1}.jpg"
  img = Image.open(img_path).convert('RGB')
  embed = new_model(data_transform(img).cuda())
  # image = image + (data_transform(img),)
  output = output + (embed,)
  img.close()

image = torch.stack(output)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900


In [ ]:
print(image.size())

torch.Size([7000, 512, 7, 7])


In [ ]:
# output = new_model(image.cuda())
torch.save(image, 'image_embedding_VGG_finetuned.pt')

RuntimeError: ignored